ДЗ - https://keras.io/examples/cifar10_cnn/ данную сеть можно взять как исходную. В ней необходимо заменить аугментации на albumentations и подобрать их правильно. Саму структуру сети тоже можно менять - точность ожидается не менее 90

In [ ]:
import numpy as np

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
from albumentations import (HorizontalFlip,
    IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)

In [ ]:
batch_size = 40
num_classes = 10
epochs = 1

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (16, 16), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (8, 8)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Результат без аугментации

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
def strong_aug(p=1):
    return Compose([
        RandomRotate90(p=0.1),
        Flip(p=1),
        Transpose(p=0.4),
        OneOf([
            IAAAdditiveGaussianNoise(p=0.9),
            GaussNoise(p=0.9),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.1),
            MedianBlur(blur_limit=1, p=0.4),
            Blur(blur_limit=3, p=0.9),
        ], p=0.2),
        ShiftScaleRotate(
            shift_limit=0.0625, 
            scale_limit=0.1, 
            rotate_limit=45, 
            p=0.4),
        OneOf([
            OpticalDistortion(p=0.9),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.1),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=10),
            IAASharpen(p=0.1),
            IAAEmboss(p=0.1),
            RandomBrightnessContrast(contrast_limit=0.9),            
        ], p=0.9),
        HueSaturationValue(p=0.9),
    ], p=p)

In [ ]:
aug = strong_aug()

In [ ]:
x_train_augmented = np.array([aug(image=np.uint8(i))['image'] for i in x_train])

In [ ]:
x_train.shape

In [ ]:
x_train_augmented.shape

In [ ]:
_x_train = np.concatenate((x_train, x_train_augmented), axis=0)

In [ ]:
_y_train = np.concatenate((y_train, y_train), axis=0)

In [ ]:
model.fit(_x_train, _y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

# Результат с аугментацией

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])